In [2]:
import pandas as pd
from scipy import stats

In [3]:
# Load the gene expression matrix CSV
expression_df = pd.read_csv('standardized_file.csv')

In [4]:
# Load the Excel file containing sample metadata
metadata_df = pd.read_excel('20130606_sample_info.xlsx', sheet_name=0)

/usr/lib/python3/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
# Melt the expression DataFrame
id_vars = ['Gene ID', 'Gene Name']  # Columns to keep as identifiers
value_vars = expression_df.columns[2:]  # Sample ID columns

expression_melted_df = expression_df.melt(
    id_vars=id_vars, value_vars=value_vars, var_name='Sample', value_name='Expression'
)

merged_df = pd.merge(expression_melted_df, metadata_df, on='Sample', how='inner')
merged_df['Expression'] = pd.to_numeric(merged_df['Expression'], errors='coerce')

In [13]:
merged_df

,Gene ID,Gene Name,Sample,Expression,Family ID,Population,Population Description,Gender,Relationship,Unexpected Parent/Child,Non Paternity,Siblings,Grandparents,Avuncular,Half Siblings,Unknown Second Order,Third Order,Other Comments
0,ENSG00000000003,TSPAN6,HG00096,NaN,HG00096,GBR,British in England and Scotland,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ENSG00000000005,TNMD,HG00096,NaN,HG00096,GBR,British in England and Scotland,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENSG00000000419,DPM1,HG00096,50.0,HG00096,GBR,British in England and Scotland,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ENSG00000000457,SCYL3,HG00096,4.0,HG00096,GBR,British in England and Scotland,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ENSG00000000460,C1orf112,HG00096,4.0,HG00096,GBR,British in England and Scotland,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22625521,ENSG00000285990,NBEAP6,NA19257,NaN,Y092,YRI,"Yoruba in Ibadan, Nigeria",female,mother,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22625522,ENSG00000285991,ENSG00000285991,NA19257,NaN,Y092,YRI,"Yoruba in Ibadan, Nigeria",female,mother,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22625523,ENSG00000285992,ENSG00000285992,NA19257,NaN,Y092,YRI,"Yoruba in Ibadan, Nigeria",female,mother,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22625524,ENSG00000285993,ENSG00000285993,NA19257,NaN,Y092,YRI,"Yoruba in Ibadan, Nigeria",female,mother,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Count unique populations
unique_populations = merged_df['Population'].unique()
num_populations = len(unique_populations)

print(f"Number of populations: {num_populations}")
print(f"Populations: {unique_populations}")

Number of populations: 5
Populations: ['GBR' 'FIN' 'TSI' 'CEU' 'YRI']


In [16]:
# Count unique individuals per population
individuals_per_population = merged_df.groupby('Population')['Sample'].nunique()

print("Unique individuals per population:")
print(individuals_per_population)
print("Sum of individuals mentioned above:")
print(sum(individuals_per_population))

Unique individuals per population:
Population
CEU    91
FIN    95
GBR    94
TSI    93
YRI    89
Name: Sample, dtype: int64
Sum of individuals mentioned above:
462


In [ ]:
# Calculate average gene expression per population
avg_expression_per_population = merged_df.groupby(['Gene Name', 'Population'])['Expression'].mean().unstack()

print("Average gene expression per population:")
(avg_expression_per_population)

Average gene expression per population:
Population        CEU        FIN        GBR        TSI        YRI
Gene Name                                                        
5S_rRNA      0.953846   0.916667   0.981818   1.438889   0.841667
5_8S_rRNA    0.500000        NaN        NaN   0.700000        NaN
7SK          0.442857   0.457143   0.433333   0.360000   0.433333
A1BG         0.113725   0.112963   0.110169   0.103636   0.117742
A1BG-AS1     1.032967   0.958947   1.139362   1.003226   0.950562
...               ...        ...        ...        ...        ...
ZYX         22.494505  22.052632  25.287234  26.666667  27.516854
ZYXP1             NaN        NaN        NaN   0.500000   0.500000
ZZEF1        4.945055   5.968421   5.989362   5.838710   7.191011
ZZZ3         5.296703   5.831579   5.904255   5.677419   5.539326
snoZ196      1.431818   1.929091   2.011475   1.738776   1.472093

[48190 rows x 5 columns]


In [10]:
# Calculate the range (max - min) of expression for each gene across populations
gene_variation = avg_expression_per_population.max(axis=1) - avg_expression_per_population.min(axis=1)

# Sort genes by variation (descending)
sorted_gene_variation = gene_variation.sort_values(ascending=False)

print("Genes with largest variation among populations:")
print(sorted_gene_variation.head(10)) # Display the top 10 genes

Genes with largest variation among populations:
Gene Name
IGKC       5029.642221
IGHM       4280.852863
MT-CO2     2634.050624
MT-ND4     2440.312755
MT-CO1     2179.535992
MT-CO3     1888.301519
HLA-DRA    1376.322188
MT-ND6     1316.678849
MT-ATP6    1265.567107
MT-RNR2    1192.356093
dtype: float64


In [12]:
# Calculate average gene expression per gender
avg_expression_per_gender = merged_df.groupby(['Gene Name', 'Gender'])['Expression'].mean().unstack()

print("Average gene expression per gender:")
print(avg_expression_per_gender)

# Calculate the difference (max - min) of expression for each gene between genders
gender_variation = abs(avg_expression_per_gender['male'] - avg_expression_per_gender['female'])

# Sort genes by variation (descending)
sorted_gender_variation = gender_variation.sort_values(ascending=False)

print("Genes with largest difference between genders:")
print(sorted_gender_variation.head(10)) # Display the top 10 genes

Average gene expression per gender:
Gender        female       male
Gene Name                      
5S_rRNA     0.900000   1.202564
5_8S_rRNA        NaN   0.600000
7SK         0.443750   0.408333
A1BG        0.112752   0.110606
A1BG-AS1    0.996341   1.041667
...              ...        ...
ZYX        24.723577  24.842593
ZYXP1       0.500000        NaN
ZZEF1       5.918699   6.050926
ZZZ3        5.617886   5.694444
snoZ196     1.834454   1.669173

[48190 rows x 2 columns]
Genes with largest difference between genders:
Gene Name
IGHM        991.598126
IGKC        713.133017
MT-ND4      441.858740
RPS4X       389.666102
MT-CO2      279.725384
MT-RNR2     276.141373
IGHV5-51    236.589901
IGLV3-19    216.887638
IGHV3-23    189.478061
MT-CO3      164.144309
dtype: float64
